In [3]:
import os
import pandas as pd
import json

# 設定 CSV 資料夾
CSV_FOLDER = 'csv_data'
output_html = 'csv_show.html'

# 收集 CSV 檔案資料
csv_files = [f for f in os.listdir(CSV_FOLDER) if f.endswith('.csv')]
all_routes = {}

for csv_file in csv_files:
    file_path = os.path.join(CSV_FOLDER, csv_file)
    df = pd.read_csv(file_path)
    coords = df[['latitude', 'longitude', 'elevation']].values.tolist()
    all_routes[csv_file] = coords

# 將所有路線資料轉為 JSON
routes_json = json.dumps(all_routes)

# HTML 輸出內容
html_content = f"""
<!DOCTYPE html>
<html lang="zh-Hant">
<head>
    <meta charset="UTF-8">
    <title>CSV 路線地圖顯示</title>
    <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
    <style>
        #map {{ height: 600px; }}
        .checkboxes {{ 
            margin: 10px 0; 
            padding: 10px;
            background-color: #f9f9f9;
            border: 1px solid #ddd;
            border-radius: 5px;
        }}
        .checkboxes label {{
            display: inline-block;
            margin-right: 15px;
            cursor: pointer;
        }}
        .checkboxes input {{
            margin-right: 5px;
        }}
    </style>
</head>
<body>
    <h2>多重 CSV 路線顯示</h2>
    <div class="checkboxes" id="checkboxContainer"></div>
    <div id="map"></div>

    <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
    <script>
        const routesData = {routes_json};
        const map = L.map('map'); // 先不設定 view，讓 fitBounds 自動調整
        L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
            maxZoom: 19,
            attribution: 'Map data © OpenStreetMap contributors'
        }}).addTo(map);

        const checkboxContainer = document.getElementById('checkboxContainer');
        const routeLayers = {{}};
        const allLatlngs = [];

        // 隨機顏色產生器
        const getRandomColor = () => {{
            const letters = '0123456789ABCDEF';
            let color = '#';
            for (let i = 0; i < 6; i++) {{
                color += letters[Math.floor(Math.random() * 16)];
            }}
            return color;
        }};

        // 建立 Checkbox 與圖層
        for (const [filename, coords] of Object.entries(routesData)) {{
            const routeColor = getRandomColor();
            
            const checkbox = document.createElement('input');
            checkbox.type = 'checkbox';
            checkbox.id = filename;
            checkbox.checked = true;

            const label = document.createElement('label');
            label.htmlFor = filename;
            // 讓標籤也帶有路線顏色
            label.innerHTML = `<span style="color: ${{routeColor}}; font-weight: bold;">&#9632;</span> ${{filename}}`;

            checkbox.addEventListener('change', () => {{
                if (checkbox.checked) {{
                    map.addLayer(routeLayers[filename]);
                }} else {{
                    map.removeLayer(routeLayers[filename]);
                }}
            }});

            checkboxContainer.appendChild(checkbox);
            checkboxContainer.appendChild(label);

            // *** 修正核心 ***
            // 1. 建立一個 Layer Group 來存放線條和所有標記點
            const routeGroup = L.layerGroup();

            // 2. 建立線條 (Polyline) 並加入 Group
            const latlngs = coords.map(c => {{
                const latlng = [c[0], c[1]];
                allLatlngs.push(latlng); // 收集所有點以便後續自動縮放
                return latlng;
            }});
            const polyline = L.polyline(latlngs, {{ color: routeColor, weight: 3 }})
                             .bindPopup(`路線: ${{filename}}`);
            polyline.addTo(routeGroup);

            // 3. 建立標記點 (CircleMarker) 並加入 Group
            coords.forEach(c => {{
                L.circleMarker([c[0], c[1]], {{
                    radius: 3,
                    color: routeColor,
                    fillColor: '#FFF',
                    fillOpacity: 0.8
                }}).bindPopup(`緯度: ${{c[0]}}<br>經度: ${{c[1]}}<br>海拔: ${{c[2]}} m`)
                  .addTo(routeGroup);
            }});

            // 將整個 Group 存起來並加到地圖上
            routeLayers[filename] = routeGroup;
            routeGroup.addTo(map);
        }}

        // 自動調整視野
        if (allLatlngs.length > 0) {{
            map.fitBounds(allLatlngs);
        }} else {{
            map.setView([23.5, 121], 8); // 如果沒有資料，顯示台灣全島
        }}
    </script>
</body>
</html>
"""

# 寫入 HTML
with open(output_html, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"地圖已生成：{output_html}")


地圖已生成：csv_show.html
